Car Pipeline:
Car Detection (YOLO) -> detect and classify driver and seatbelt (Custom) Not Done Yet -> license plate detection(YOLO) -> OCR (pytesseract)


In [ ]:
import sys
import os 
if 'google.colab' in sys.modules:
    !pip -q install ultralytics
    !pip -q install pytesseract
    !apt-get -qq install tesseract-ocr
    if not os.path.exists('license-plate-finetune-v1s.pt'):
        !wget https://huggingface.co/morsetechlab/yolov11-license-plate-detection/resolve/main/license-plate-finetune-v1s.pt


In [ ]:
from ultralytics import YOLO
from ultralytics.engine.results import Results
from pathlib import Path
import pytesseract
from PIL import Image
import numpy as np
from typing import Any

license_plate_detection_model = YOLO('license-plate-finetune-v1s.pt')

car_detection_model = YOLO("yolo11n.pt")


In [ ]:
def detect_cars(image_path: Path | str) -> list[np.ndarray]:
    # First, detect cars in the image
    car_results = car_detection_model.predict(source=image_path, classes=[2,5,7], verbose=False)  # COCO classes for car, bus, truck
    output = []
    for car in car_results[0].boxes:
        # Extract car bounding box
        x1, y1, x2, y2 = map(int, car.xyxy[0])
        car_image = car_results[0].orig_img[y1:y2, x1:x2]
        output.append(car_image)  
    return output

In [ ]:
def plate_detection(car_images: list[np.ndarray]) -> list[Results]:
    # don't output no detections
    plate_results = license_plate_detection_model.predict(source=car_images, verbose=False,conf=0.25)
    return plate_results

In [ ]:
def get_plate_images(plate_results: list[Results]) -> list[np.ndarray]:
    output = []
    for plate in plate_results:
        for box in plate.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            plate_image = plate.orig_img[y1:y2, x1:x2]
            output.append(plate_image)
    return output   

In [ ]:
def run_ocr_on_plate(plate_images: list[np.ndarray])->list[str]:
    ocr_results = []
    for plate in plate_images:
        
        # Convert numpy array to PIL Image for pytesseract
        pil_image = Image.fromarray(plate)
        ocr_result = pytesseract.image_to_string(pil_image, config='--psm 7')
        ocr_results.append(ocr_result.strip())
    return ocr_results


In [ ]:
def run_pipeline(image_path: Path | str):
    car_images = detect_cars(image_path)
    plate_results = plate_detection(car_images)
    plate_images = get_plate_images(plate_results)
    return plate_images,plate_results

In [ ]:
if 'google.colab' in sys.modules:
    dirl = os.listdir(".")
    if "bmwalleng.png" not in dirl:
      !wget https://github.com/AFAskar/dlimageclass/blob/main/bmwalleng.png?raw=true
      !mv bmwalleng.png?raw=true bmwalleng.png
    if '2cars.png' not in dirl:
      !wget https://github.com/AFAskar/dlimageclass/blob/main/2cars.png?raw=true
      !mv 2cars.png?raw=true 2cars.png


In [ ]:


import json

images = ['./bmwalleng.png','./2cars.png']
all_results = []
for img_path in images:
    plate_images, plate_results = run_pipeline(img_path)
    ocr_results = run_ocr_on_plate(plate_images)    
    all_results.extend(plate_results)

for plate in all_results:
    plate.show()
with open('ocr_results.json', 'w') as f:
    json.dump(ocr_results, f)